In [29]:
import numpy as np 
import pandas as pd 

import sklearn
from sklearn.linear_model import Lasso
from tensorflow.python.client import device_lib

import numpy as np
import pandas as pd
import sklearn
from keras.utils import to_categorical


#input csv file
training_data = pd.read_csv('training.csv', delimiter = ',') #training data frame


#parse out annoying strings in front 
#training_data['segment_id'] =  training_data['segment_id'].apply(lambda x: int(x[10:]))
training_data['wellbore_chev_no_id'] =  training_data['wellbore_chev_no_id'].apply(lambda x: int(x[17:]))
training_data['area_id'] =  training_data['area_id'].apply(lambda x: int(x[5:]))
training_data['formation_id'] =  training_data['formation_id'].apply(lambda x: int(x[10:]))
training_data['bit_model_id'] =  training_data['bit_model_id'].apply(lambda x: int(x[10:]))

#training_data.to_csv('Clean Data.csv')

print (training_data.nunique())

#print (training_data)

#print (training_data.isnull().sum(axis = 0)) #no null values 

from sklearn.datasets import load_iris
from sklearn import preprocessing


y = np.array(training_data['rate_of_penetration'])
#y = preprocessing.scale(y)
X = np.array(training_data.drop(['segment_id','rate_of_penetration'], axis=1))


from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from keras import backend
 

#one-hot encoding
X = pd.get_dummies(data=training_data, columns=['wellbore_chev_no_id', 'area_id', 'formation_id', 'bit_model_id', 'drillbit_size'])
cols_to_norm = ['min_depth','max_depth', 'surface_weight_on_bit', 'surface_rpm']
X[cols_to_norm] = X[cols_to_norm].apply(lambda x: preprocessing.scale(x)) #only standardize non-one-hot columns
#print (X)
X = np.array(X.drop(['segment_id','max_depth', 'rate_of_penetration'], axis=1))
#print(pd.get_dummies(training_data['wellbore_chev_no_id']))
#print (X.nunique())


#Models

#grid search for parameters
from sklearn import metrics
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
%matplotlib inline
from matplotlib import pyplot as plt
from keras import optimizers #try adam and sgd
from sklearn.metrics import mean_squared_error

import numpy as np
from sklearn.model_selection import train_test_split
import math
#for a in [0.01, 0.05, 0.1, 0.5, 1, 5]:
    #for m in [100,500,1000,5000,10000]:
    
    
def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))


#split training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42) #split data (85% train, 15% test)


model1 = Sequential()
model1.add(Dense(2, input_dim=310, activation='relu'))
model1.add(Dense(5, kernel_initializer='normal',activation='relu'))
model1.add(Dense(5, kernel_initializer='normal',activation='relu'))
model1.add(Dense(1, kernel_initializer = 'normal', activation = 'linear'))

#IMPORTANT - use this if you want to train, and comment out load weights
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model1.compile(loss='mse', optimizer='adam', metrics=[rmse])
#history = model1.fit(X_train, y_train, validation_split=0.15, epochs=500, batch_size=100, verbose=2)
#model1.save_weights("NN.h5")


#IMPORTANT - load pre-existing weights for best model, no need to train
model1.load_weights('NN.h5')

model2 = Sequential()
model2.add(Dense(2, input_dim=310, activation='relu'))
model2.add(Dense(7, kernel_initializer='normal',activation='relu'))
model2.add(Dense(7, kernel_initializer='normal',activation='relu'))
model2.add(Dense(1, kernel_initializer = 'normal', activation = 'linear'))
model2.compile(loss='mse', optimizer='adam', metrics=[rmse])
#history = model2.fit(X_train, y_train, validation_split=0.15, epochs=500, batch_size=150, verbose=2)
#model2.save_weights("NN2.h5")

model2.load_weights('NN2.h5')


model3 = Sequential()
model3.add(Dense(2, input_dim=310, activation='relu'))
model3.add(Dense(6, kernel_initializer='normal',activation='relu'))
model3.add(Dense(6, kernel_initializer='normal',activation='relu'))
model3.add(Dense(1, kernel_initializer = 'normal', activation = 'linear'))
model3.compile(loss='mse', optimizer='adam', metrics=[rmse])
#history = model3.fit(X_train, y_train, validation_split=0.15, epochs=500, batch_size=150, verbose=2)
#model3.save_weights("NN3.h5")

model3.load_weights('NN3.h5')


model4 = Sequential()
model4.add(Dense(2, input_dim=310, activation='relu'))
model4.add(Dense(8, kernel_initializer='normal',activation='relu'))
model4.add(Dense(8, kernel_initializer='normal',activation='relu'))
model4.add(Dense(1, kernel_initializer = 'normal', activation = 'linear'))
model4.compile(loss='mse', optimizer='adam', metrics=[rmse])
#history = model4.fit(X_train, y_train, validation_split=0.15, epochs=500, batch_size=150, verbose=2)
#model4.save_weights("NN4.h5")

#try different activation functions
#different optimizers 

model4.load_weights('NN4.h5')

import pickle

"""
params = {'n_estimators': 500, 'max_depth': 5, 'min_samples_split': 3, #3 samples fit, 250/10 vs 500/5
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf.fit(X_train, y_train)
filename = 'GBoost1_m.sav'
pickle.dump(clf, open(filename, 'wb'))


params2 = {'n_estimators': 100, 'max_depth': 15, 'min_samples_split': 3, #3 samples fit, 250/10 vs 500/5
          'learning_rate': 0.01, 'loss': 'ls'}
clf2 = ensemble.GradientBoostingRegressor(**params2)
clf2.fit(X_train, y_train)
filename = 'GBoost2_m.sav'
pickle.dump(clf2, open(filename, 'wb'))

"""

gb1 = pickle.load(open('Gboost1.sav', 'rb'))
gb2 = pickle.load(open('Gboost2.sav', 'rb'))



y_pred5 = gb1.predict(X_test)
y_pred5 = np.reshape(y_pred5,(1026,1))
rmse_gb = math.sqrt(mean_squared_error(y_test, y_pred5))
print("RMSE: %.4f" % rmse_gb)


y_pred6 = gb2.predict(X_test)
y_pred6 = np.reshape(y_pred6,(1026,1))
rmse_gb = math.sqrt(mean_squared_error(y_test, y_pred6))
print("RMSE: %.4f" % rmse_gb)




"""
use this to calc rmse between two vectors of the same length
"""

def rmse_non_tensor(a,b):
    total = 0
    for i in range(len(a)):
        total += (a[i]-b[i])**2
    total = total/len(a)
    total = math.sqrt(total)
    return total

y_pred1 = model1.predict(X_test) #model prediction on X test set
y_pred2 = model2.predict(X_test) #model prediction on X test set
y_pred3 = model3.predict(X_test) #model prediction on X test set
y_pred4 = model4.predict(X_test) #model prediction on X test set


#Ensemble of 4 ANNs + Gradient Boost Decision Tree Emsemble Regressor
y_pred = ((0.1*(y_pred1 + y_pred2 + y_pred3 + y_pred4 + y_pred5) + 0.5*y_pred6))

#y_pred = ((y_pred5 + y_pred6)/2.0)
y_test = np.reshape(y_test,(1026,1))
#print (y_pred.shape)
#print (y_test.shape)

print (rmse_non_tensor(y_pred,y_test))


final = np.concatenate((y_test, y_pred), axis=1) #test is actual, pred is prediction

final = pd.DataFrame(final)
final.columns = ['Actual', 'Prediction']

print(final)
final = final.drop(['Actual'], axis = 1)
export_csv = final.to_csv ('Prediction.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path



"""
#summarize history for rmse
plt.plot(history.history['rmse'])
plt.plot(history.history['val_rmse'])
plt.title('Model RMSE')
plt.ylabel('RMSE')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


#cross val

from sklearn.model_selection import KFold #K fold cross validation 
#X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
#y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=10)
kf.get_n_splits(X)

for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
   
   #lsvc = linear_model.Lasso(alpha=1, max_iter = 1000).fit(X_train, y_train)
    #y_pred = lsvc.predict(X_test)
    #regressor = RandomForestRegressor(n_estimators=100, max_depth = 15, random_state=0)
    #regressor.fit(X_train, y_train)
    #y_pred = regressor.predict(X_test)
    #print (y_pred)
    #print (y_test)
    #print (np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    
    
    model = Sequential()
    model.add(Dense(2, input_dim=311, activation='linear'))
    model.add(Dense(5, kernel_initializer='normal',activation='linear'))
    model.add(Dense(5, kernel_initializer='normal',activation='linear'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam', metrics=[rmse])
    history = model.fit(X_train, y_train, epochs=500, batch_size=100, verbose=2)

"""    





segment_id               6838
wellbore_chev_no_id        21
area_id                     2
formation_id              239
bit_model_id               38
drillbit_size               7
min_depth                 990
max_depth                1542
rate_of_penetration      6838
surface_weight_on_bit    6837
surface_rpm              6210
dtype: int64
RMSE: 21.3100
RMSE: 18.8120
19.592554397507538
          Actual  Prediction
0     227.103109  247.493798
1      78.116084   67.972360
2      55.749590   63.659276
3      46.947824   56.502388
4     163.160333  165.257118
5     105.309330  105.530433
6     115.418377  123.424452
7     138.143219  136.061614
8     102.287113  105.429853
9      77.027467   59.092847
10    120.285998  110.343820
11     78.117781   80.421242
12     71.308891   84.721874
13     58.593417   68.198066
14     43.936250   73.646898
15    191.948166  182.265445
16     99.284859   87.001610
17    121.356377  117.738209
18     89.865083   94.361298
19    131.967490  122.050480
2

'\n#summarize history for rmse\nplt.plot(history.history[\'rmse\'])\nplt.plot(history.history[\'val_rmse\'])\nplt.title(\'Model RMSE\')\nplt.ylabel(\'RMSE\')\nplt.xlabel(\'Epoch\')\nplt.legend([\'train\', \'validation\'], loc=\'upper left\')\nplt.show()\n\n# summarize history for loss\nplt.plot(history.history[\'loss\'])\nplt.plot(history.history[\'val_loss\'])\nplt.title(\'Model Loss\')\nplt.ylabel(\'Loss\')\nplt.xlabel(\'Epoch\')\nplt.legend([\'train\', \'validation\'], loc=\'upper left\')\nplt.show()\n\n\n#cross val\n\nfrom sklearn.model_selection import KFold #K fold cross validation \n#X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])\n#y = np.array([1, 2, 3, 4])\nkf = KFold(n_splits=10)\nkf.get_n_splits(X)\n\nfor train_index, test_index in kf.split(X):\n    #print("TRAIN:", train_index, "TEST:", test_index)\n    X_train, X_test = X[train_index], X[test_index]\n    y_train, y_test = y[train_index], y[test_index]\n   \n   #lsvc = linear_model.Lasso(alpha=1, max_iter = 1000).fit(X_trai